In [1]:
import wrangle


Importing data from csv file and reading to df using `wrangle.py` function

In [2]:
df = wrangle.get_curriculum_logs_data()
df.head()

The df has 847330 rows and 10 columns.


,cohortid_cohorts,start_date,end_date,cohortname,program_id,date,time,path,user_id,ip
0,1,2014-02-04,2014-04-22,Arches,1,2018-01-26,10:14:47,/,11,97.105.19.61
1,1,2014-02-04,2014-04-22,Arches,1,2018-01-26,10:14:51,javascript-i,11,97.105.19.61
2,1,2014-02-04,2014-04-22,Arches,1,2018-01-26,10:15:02,javascript-i/functions,11,97.105.19.61
3,1,2014-02-04,2014-04-22,Arches,1,2018-01-26,11:46:29,appendix/further-reading/javascript/hoisting.html,11,97.105.19.61
4,1,2014-02-04,2014-04-22,Arches,1,2018-01-26,11:46:39,appendix,11,97.105.19.61


Getting familiar with the data...

In [3]:
print(f'There is {df.isna().sum().sum()} null value(s).')
print()
print(df.info())

There is 1 null value(s).

<class 'pandas.core.frame.DataFrame'>
Int64Index: 847330 entries, 0 to 847329
Data columns (total 10 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   cohortid_cohorts  847330 non-null  int64 
 1   start_date        847330 non-null  object
 2   end_date          847330 non-null  object
 3   cohortname        847330 non-null  object
 4   program_id        847330 non-null  int64 
 5   date              847330 non-null  object
 6   time              847330 non-null  object
 7   path              847329 non-null  object
 8   user_id           847330 non-null  int64 
 9   ip                847330 non-null  object
dtypes: int64(3), object(7)
memory usage: 71.1+ MB
None


#### First look issues
- Need to investigate the x1 null value
- `start_date`, `end_date`, `date`, and `time` need to be datetype dtype
- `date` needs to be set as index col
- I want to create a `days_since_end` column showing the difference between the last curriculum access `date` and the cohort's end date `end_date`
- map the `prodgram_id` col to the actual program name
- drop the following cols:
    - `cohortid_cohorts`
    - `program_id`

In [7]:
# investigating the x1 null value
df[df.path.isna()]

,cohortid_cohorts,start_date,end_date,cohortname,program_id,date,time,path,user_id,ip
562633,55,2020-02-03,2020-07-07,Curie,3,2020-04-08,09:25:18,NaN,586,72.177.240.51


What I know about the x1 null:
- For this log entry, the curriculum was accessed between cohort start and end dates, in the morning
- This was a user from the `Curie` cohort
- The null log was from user `586`

In [20]:
df[(df.cohortname == 'Curie') & (df.date == '2020-04-08') & (df.time <= '09:30:00') & (df.user_id == 586)]\
                                            .sort_values(by = 'time')

,cohortid_cohorts,start_date,end_date,cohortname,program_id,date,time,path,user_id,ip
562614,55,2020-02-03,2020-07-07,Curie,3,2020-04-08,09:20:22,6-regression/8.0-Project,586,72.177.240.51
562620,55,2020-02-03,2020-07-07,Curie,3,2020-04-08,09:20:43,6-regression/8.0-Project,586,72.177.240.51
562630,55,2020-02-03,2020-07-07,Curie,3,2020-04-08,09:24:37,6-regression/8.0-Project,586,72.177.240.51
562632,55,2020-02-03,2020-07-07,Curie,3,2020-04-08,09:25:09,/,586,72.177.240.51
562633,55,2020-02-03,2020-07-07,Curie,3,2020-04-08,09:25:18,NaN,586,72.177.240.51
562634,55,2020-02-03,2020-07-07,Curie,3,2020-04-08,09:25:18,2-storytelling/2.2-create,586,72.177.240.51
562638,55,2020-02-03,2020-07-07,Curie,3,2020-04-08,09:25:53,/,586,72.177.240.51
562639,55,2020-02-03,2020-07-07,Curie,3,2020-04-08,09:26:13,6-regression/8-project,586,72.177.240.51


Since the null value is only a tiny fraction of the total data will drop for now. In second look can also check what other cohort-mates were looking at around this time +/- 5 or 10m and take the mode and fill the na.